<a href="https://colab.research.google.com/github/dmi3eva/araneae/blob/main/p1_spider_analysis/_UTILS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content
%cd drive
%cd My\ Drive
%cd PhD
%cd Paper_01

/content
/content/drive
/content/drive/My Drive
/content/drive/My Drive/PhD
/content/drive/My Drive/PhD/Paper_01


In [4]:
import json
import collections
import matplotlib.pyplot as plt
import sqlite3
import os

### Обращение к БД

Вспомогательное

In [5]:
DB_PATH = "datasets/spider/database"
SCHEMES_PATH = "datasets/spider/tables.json"

In [6]:
def execute_request(db_id, sql):
    db = os.path.join(DB_PATH, db_id, db_id + ".sqlite")
    conn = sqlite3.connect(db)
    conn.text_factory = lambda b: b.decode(errors = 'ignore')
    cursor = conn.cursor()   
    try:
        cursor.execute(sql)
        res = cursor.fetchall()
        return res
    except:
        raise ValueError

In [7]:
REQUEST_MASK = """SELECT \"{column}\" FROM \"{table}\"""" 
def get_values(db, table, column):
  # Возвращает значения из столбца данной таблицы
  aim_request = REQUEST_MASK.format(table=table, column=column)
  print(aim_request)
  try:
    response = execute_request(db, aim_request)
    values = [str(_v[0]) for _v in list(response)]
    return values
  except ValueError:
    print()
    print(f"Problem with {column} in {table} (db = {db}). Request: {aim_request}")
    return []

In [8]:
db_ids = os.listdir(DB_PATH)

In [9]:
with open(SCHEMES_PATH) as table_file:
  schemes = json.load(table_file)

tables = {_s['db_id']: _s['table_names_original'] for _s in schemes}
sorted(list(tables.keys()))

['academic',
 'activity_1',
 'aircraft',
 'allergy_1',
 'apartment_rentals',
 'architecture',
 'assets_maintenance',
 'baseball_1',
 'battle_death',
 'behavior_monitoring',
 'bike_1',
 'body_builder',
 'book_2',
 'browser_web',
 'candidate_poll',
 'car_1',
 'chinook_1',
 'cinema',
 'city_record',
 'climbing',
 'club_1',
 'coffee_shop',
 'college_1',
 'college_2',
 'college_3',
 'company_1',
 'company_employee',
 'company_office',
 'concert_singer',
 'county_public_safety',
 'course_teach',
 'cre_Doc_Control_Systems',
 'cre_Doc_Template_Mgt',
 'cre_Doc_Tracking_DB',
 'cre_Docs_and_Epenses',
 'cre_Drama_Workshop_Groups',
 'cre_Theme_park',
 'csu_1',
 'culture_company',
 'customer_complaints',
 'customer_deliveries',
 'customers_and_addresses',
 'customers_and_invoices',
 'customers_and_products_contacts',
 'customers_campaigns_ecommerce',
 'customers_card_transactions',
 'debate',
 'decoration_competition',
 'department_management',
 'department_store',
 'device',
 'document_management',

In [10]:
columns = {
    _s['db_id']: {
        _table: [] for _table in _s['table_names_original']
    } for _s in schemes
}
for _scheme in schemes:
  db_id = _scheme['db_id']
  table_names = _scheme['table_names_original']
  for _column in _scheme["column_names_original"]:
    column_name = _column[1]
    table_name = table_names[_column[0]]
    if column_name == '*':
      continue
    columns[db_id][table_name].append(column_name)

In [11]:
def get_db_tables(db):
  return tables[db]

In [13]:
def get_db_columns(db, table):
  return columns[db][table]

In [54]:
def show_table(db, table):
  values_request = f"SELECT * FROM {table}"
  values = execute_request(db, values_request)  
  column_names = tuple(get_db_columns(db, table))
  return column_names, values

Сами обращения

In [57]:
db = 'apartment_rentals'
table = 'View_Unit_Status'
column = 'Claim_Status_Description'

In [58]:
result_tables = get_db_tables(db)
result_tables

['Apartment_Buildings',
 'Apartments',
 'Apartment_Facilities',
 'Guests',
 'Apartment_Bookings',
 'View_Unit_Status']

In [59]:
result_columns = get_db_columns(db, table)
result_columns

['apt_id', 'apt_booking_id', 'status_date', 'available_yn']

In [ ]:
result_values = get_values(db, table, column)
result_values

In [65]:
sql_request = "SELECT status_date FROM View_Unit_Status WHERE available_yn=0"
result = execute_request(db, sql_request)
result

[('1985-11-05 11:46:35',), ('2000-02-04 07:50:09',), ('2012-11-24 13:39:37',)]

In [61]:
table_view = show_table(db, table)
table_view

(('apt_id', 'apt_booking_id', 'status_date', 'available_yn'),
 [(11, 920, '1970-09-28 10:24:29', 1),
  (15, 575, '1972-03-23 22:55:53', 1),
  (15, 924, '1973-10-28 04:30:14', 1),
  (6, 497, '1976-12-18 04:03:51', 1),
  (12, 807, '1977-04-15 13:42:19', 1),
  (13, 575, '1978-12-28 11:53:34', 1),
  (2, 497, '1980-11-12 13:34:45', 1),
  (14, 401, '1985-11-05 11:46:35', 0),
  (11, 497, '1990-11-04 17:57:50', 1),
  (13, 337, '2000-02-04 07:50:09', 0),
  (14, 279, '2001-02-17 20:17:09', 1),
  (5, 337, '2003-07-25 10:13:48', 1),
  (1, 497, '2003-08-02 08:36:36', 1),
  (10, 497, '2006-02-23 05:50:04', 1),
  (6, 401, '2011-02-12 09:04:07', 1),
  (9, 623, '2011-11-06 22:08:42', 1),
  (14, 920, '2012-11-24 13:39:37', 0),
  (4, 258, '2014-12-10 13:53:21', 1),
  (13, 343, '2015-06-19 07:59:01', 1),
  (5, 889, '2015-07-15 11:06:29', 1)])

In [56]:
table_view = show_table(db, table)
table_view

(('Claim_Header_ID',
  'Claim_Status_Code',
  'Claim_Type_Code',
  'Policy_ID',
  'Date_of_Claim',
  'Date_of_Settlement',
  'Amount_Claimed',
  'Amount_Piad'),
 [(15,
   'Settled',
   'Handphone Subsidy',
   518,
   '2016-05-31 06:07:11',
   '2018-02-23 03:46:38',
   349.15,
   582.03),
  (24,
   'Disputed',
   'Child Birth',
   518,
   '2016-12-30 21:43:21',
   '2017-10-08 21:43:14',
   318.16,
   309.2),
  (27,
   'Disputed',
   'Overtime Meal Subsidy',
   518,
   '2017-05-01 13:34:43',
   '2017-11-16 04:06:05',
   362.71,
   132.97),
  (28,
   'Disputed',
   'Child Birth',
   287,
   '2016-10-01 00:40:00',
   '2017-10-20 02:24:21',
   789.19,
   113.86),
  (34,
   'Disputed',
   'Handphone Subsidy',
   518,
   '2016-04-10 01:36:18',
   '2017-10-17 07:02:17',
   270.49,
   643.82),
  (39,
   'In Progress',
   'Handphone Subsidy',
   518,
   '2016-09-17 05:05:07',
   '2018-01-07 05:28:53',
   616.12,
   773.5),
  (45,
   'In Progress',
   'Handphone Subsidy',
   507,
   '2016-06-12 2

## Выполнение запроса